In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from scipy import interpolate
from numpy import *

In [2]:
def write_orbit(x, y, N, outfile):
    
    with open(outfile, 'w') as f:
        
        for i in range(N):
            
            f.write("{0}, {1}\n".format(x[i], y[i]))
        

In [88]:
m = 1
M = 40
R = 1
G = 1

In [89]:
k = m/M
lamda = k/(1+k)
w = 1
rm = (1 - lamda)*R
rM = lamda*R

In [90]:
def S(r, phi):
    
    return ((r*sin(phi))**2 + (r*cos(phi) + lamda*R)**2)**0.5

def s(r, phi):
    
    return ((r*sin(phi))**2 + (r*cos(phi) - (1 - lamda)*R)**2)**0.5

In [91]:
def rdot(t, r, phi, p, l):
    
    return p

def phidot(t, r, phi, p, l):
    
    return (l/(r**2)) - w

def pdot(t, r, phi, p, l):
    
    return (l**2/r**3) - (w**2)*(R**3)*(1-lamda)*(r + lamda*R*cos(phi))/(S(r,phi))**3 - w**2*R**3*lamda*(r - (1 - lamda)*R*cos(phi))/(s(r,phi))**3

def ldot(t, r, phi, p, l):
    
    return w**2*R**4*lamda*(1-lamda)*(1/(S(r,phi))**3 - 1/(s(r,phi))**3)*r*sin(phi)

In [92]:
def fdot(v, t):
    
    r = v[0]
    phi = v[1]
    p = v[2]
    l = v[3]
    
    return array([rdot(t, r, phi, p, l), phidot(t,r,phi,p,l), pdot(t,r,phi,p,l), ldot(t,r,phi,p,l)], float)

In [93]:

#h = (tmax - t0)/N
#t = arange(t0, tmax, h)
r = []
phi = []
p = []
l = []

In [94]:
def rk(x0, y0, outfile):
    
    t0 = 0
    tmax = 100
    N = 100000
    
    h = (tmax - t0)/N
    tpoints = arange(t0, tmax, h)
    r = []
    phi = []
    p = []
    l = []
    
    r0 = sqrt(x0**2 + y0**2)
    phi0 = arctan(y0/x0)
    
    v = array([r0, phi0, 0, 0])
    
    for t in tpoints:
        
        r.append(v[0])
        phi.append(v[1])
        p.append(v[2])
        l.append(v[3])
        
        k1 = h*fdot(v, t)
        
        k2 = h*fdot(v+0.5*k1, t+0.5*h)
        
        k3 = h*fdot(v+0.5*k2, t+0.5*h)
        
        k4 = h*fdot(v+k3, t+h)
        
        v += (k1 + 2*k2 + 2*k3 + k4)/6
        
    r = array(r)
    phi = array(phi)
    
    x = r*cos(phi)
    y = r*sin(phi)
            
    write_orbit(x,y, N, outfile)

In [95]:
#L4
x4 = (R/2) * (M - m)/(M + m)
y4 = R*sin(pi/3)

#rk(x4,y4, "l4.csv")

#L5
x5 = x4
y5 = -y4
#rk(x5, y5, "l5.csv")

In [96]:
from scipy.integrate import solve_ivp
from scipy.integrate import odeint

In [97]:
def fdot(t,v):
    
    r, phi, p, l = v
    
    dvdt = [rdot(t, r, phi, p, l), phidot(t,r,phi,p,l), pdot(t,r,phi,p,l), ldot(t,r,phi,p,l)]
    
    return dvdt

In [177]:
tmax = 1.5
N = 1000
t = linspace(0, tmax, N)

In [182]:
r40 = sqrt(x4**2 + y4**2)
phi40 = arctan(-y4/x4)

v40 = [r40, phi40, 0, 0]

In [183]:
sol = solve_ivp(fdot, [0, tmax], v40, "BDF")

In [184]:
x = sol.y[0] * cos(sol.y[1])
y = sol.y[0] * sin(sol.y[1])

In [185]:
write_orbit(x,y,len(x),"l4.csv", )

In [186]:
sol

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 675
     njev: 22
      nlu: 67
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.42275357e-04, 2.84550714e-04, 1.70730428e-03,
       3.13005785e-03, 1.43560995e-02, 2.55821412e-02, 3.68081829e-02,
       8.33041630e-02, 1.29800143e-01, 1.76296123e-01, 2.81378925e-01,
       3.86461727e-01, 4.91544529e-01, 5.96627331e-01, 7.18001908e-01,
       8.02215816e-01, 8.86429724e-01, 9.33582783e-01, 9.80735841e-01,
       1.01261179e+00, 1.03334339e+00, 1.04855002e+00, 1.06375664e+00,
       1.07293428e+00, 1.07867156e+00, 1.08320694e+00, 1.08623014e+00,
       1.08841686e+00, 1.09060357e+00, 1.09122266e+00, 1.09184174e+00,
       1.09246083e+00, 1.09307991e+00, 1.09347259e+00, 1.09374746e+00,
       1.09394521e+00, 1.09407626e+00, 1.09420731e+00, 1.09427592e+00,
       1.09434453e+00, 1.09436436e+00, 1.09438419e+00, 1.09440402e+00,
       1.09442385e+00, 1.09443665e+00,

In [24]:
# r = sol[:, 0]
# phi = sol[:, 1]

In [25]:
# x = r*cos(phi)
# y = r*sin(phi)

In [26]:
# write_orbit(x,y,N,"l4.csv")